In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ProgressBar
from pytorch_lightning.callbacks import Callback


import pandas as pd
import numpy as np
import seaborn as sns
import pacmap

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import shap
import logging
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px


In [3]:
torch.set_float32_matmul_precision('medium') # Or 'high'
pio.renderers.default = 'iframe'

In [4]:
Args = type('Args', (object,), {})
args = Args()

args.file = 'data/libxml2_train.pkl'
args.inputs = 257
args.outputs = 1
args.patience = 5
args.min_delta = 0.001
args.val_size = 0.2
args.batch_size = 128
args.max_epochs = 5000
args.pacmap_samples = 10000

# Base Model and Helpers

In [5]:
class RuftModel(pl.LightningModule):
    def __init__(self, input_size, output_size, hidden_size=128, dropout=0.5):
        super().__init__()
        self.layer = nn.Linear(input_size, hidden_size)
        self.hidden = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU() 
        self.dropout = nn.Dropout(dropout)

        #self.sigmoid = nn.Sigmoid() 
        
    def forward(self, x):
        x = self.layer(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output(x)
        #x = self.sigmoid(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch  
        y_hat = self(x)
        loss = nn.BCEWithLogitsLoss()(y_hat.view(-1), y.type_as(y_hat).view(-1))  # Adjusting shapes
        self.log('loss', loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        return self.training_step(batch, batch_idx)
    
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.BCEWithLogitsLoss()(y_hat.view(-1), y.type_as(y_hat).view(-1))
        self.log('val_loss', val_loss)
        
        return val_loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [6]:
class CustomProgressBar(ProgressBar):
    def __init__(self):
        super().__init__()
        self.training_loss = []
        self.validation_loss = []
    
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        super().on_train_batch_end(trainer, pl_module, outputs, batch, batch_idx)

    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        pass
    
    def on_validation_epoch_end(self, trainer, pl_module):
        loss_txt = "N/A"
        val_loss_txt = "N/A"
        loss = trainer.logged_metrics.get('loss')
        val_loss = trainer.logged_metrics.get('val_loss')
        
        if loss is not None: 
            loss_txt = f"{loss:.4f}"
            self.training_loss.append(loss.cpu())
            
        if val_loss is not None: 
            val_loss_txt = f"{val_loss:.4f}"
            self.validation_loss.append(val_loss.cpu())    
        
        print(f'Epoch {trainer.current_epoch + 1}: loss / val_loss = {loss_txt} / {val_loss_txt}')



In [7]:
class ResourceDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float)
        self.targets = torch.tensor(targets, dtype=torch.float)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]


# Dataset loading

In [8]:
data = pd.read_pickle(args.file)  
data = data.sample(frac=1) # Make sure we've sorted
samples = data.iloc[:,-2:-1]
X = data.iloc[:, :-2].values # -1 skip label
y = data.iloc[:, -1:].values # -1 only 1 label

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=args.val_size)


In [9]:
train_dataset = ResourceDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=10, shuffle=True)
val_dataset = ResourceDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=10)

# Dataset Visualization

In [10]:
embedding = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0) 
X_transformed = embedding.fit_transform(X_val[:args.pacmap_samples], init="pca")


In [11]:
# Must convert to string or plotly will complain about bytes
sample_text = []
for s in data['sample'][:args.pacmap_samples]: sample_text.append([str(s)])


plotly_rows = np.hstack([X_transformed, y_val[:args.pacmap_samples], sample_text])
plotly_df = pd.DataFrame( plotly_rows, columns=["X", "Y", "Label", "Sample"] )

In [12]:
# Set the proper types in the dataframe
convert_dict = {'X': float,
                'Y': float,
                'Label': float
                }
 
plotly_df = plotly_df.astype(convert_dict)

In [13]:
fig = px.scatter( plotly_df, x='X' , y='Y', color='Label', custom_data=['Sample'], 
                  title="Initial Dataset PaCMAP", width=800, height=800, template = 'plotly_dark')
fig.update_traces(hovertemplate =  "%{customdata[0]}")
fig.update_xaxes(showticklabels=False) 
fig.update_yaxes(showticklabels=False)
fig.show()

# Model Training

In [14]:
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=args.min_delta,
    patience=args.patience,
    verbose=False,
    mode='min'
)

In [22]:
model = RuftModel(args.inputs, args.outputs)

torch.onnx.export(model,torch.tensor(X_train[0], dtype=torch.float),"test.onnx") #, batch.text, 'rnn.onnx', input_names=input_names, output_names=output_names)



In [ ]:
progress_bar = CustomProgressBar()

trainer = pl.Trainer(
    max_epochs=args.max_epochs,
    callbacks=[early_stop_callback, progress_bar],
    logger=True
)

startLoss = trainer.test(model, train_dataloader, verbose=False)[0]['loss']
progress_bar.training_loss.append(startLoss) # Seed the metrics with the initial loss

In [ ]:
trainer.fit(model, train_dataloader, val_dataloader);

# SHAP Values

In [ ]:
data_for_explanation = next(iter(val_dataloader))
features, _ = data_for_explanation

explainer = shap.GradientExplainer(model, features)
shap_values = explainer.shap_values(features)
shap.summary_plot(shap_values, features, feature_names=list(data.columns))

In [ ]:
exp = explainer(features)[24]
exp.base_values = 0.5
exp.feature_names = list(data.columns)

In [ ]:
shap.waterfall_plot(exp, max_display=25)

In [ ]:
sns.set_style("darkgrid")
plt.plot(progress_bar.training_loss, label='Training Loss', zorder=1)
#plt.plot(progress_bar.validation_loss, label='Validation Loss', linestyle=(0, (1, 10)))
plt.scatter( range(len(progress_bar.validation_loss)), progress_bar.validation_loss, 
             s=50, alpha=0.5, c='orange', zorder=2)
plt.plot(progress_bar.validation_loss, label='Validation Loss', c='orange', linestyle='dashed', alpha=0.25)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()